# Оценка релевантности организаций запросам на Яндекс.Картах с помощью LLM-агента

<img src="https://sun9-65.userapi.com/impg/N4y2cxlL7PauAs82tBNFOUAiNctFICWDy4Mbiw/Jiz1fb7NLWU.jpg?size=1080x1080&quality=95&sign=df2786058624d9ccac3ede4d5d056e2f&type=album" width="500" height="500" />


## Описание задачи

Необходимо создать LLM-агента, который будет оценивать релевантность организаций на Яндекс.Картах широким запросам (например, "ресторан с верандой" или "романтичный джаз-бар"). Такие запросы называются *рубричными*: пользователь здесь ищет не конкретную организацию, а идёт в Яндекс.Карты для поиска и выбора мест.

LLM-агент должен будет самостоятельно находить необходимые данные для принятия правильного решения.

Данные представлены компанией Яндекс и являются результатами асессорской разметки релевантности. Мы очень заинтересован в успешном решении задачи:)

## План работы

1. Написать сильный бейзлайн. Например, можно просто делать один запрос в LLM, возможно с добавлением в промпт размеченных примеров. Либо можно дообучить трансформер на задачу классификации.
2. Разобраться с фреймворком для реализации LLM-агентов (предлагается использовать LangGraph, оба примера в доп. материалах используют его).
3. Реализовать первую версию агента: предложить ему не принимать решение мгновенно, а ходить в поисковик для уточнения своего ответа.
4. Поработать над промптами для улучшения качества. Здесь нужно будет посмотреть, в каких запросах агент ошибается. **ПОЖАЛУЙСТА, НЕ ПОДГЛЯДЫВАЙТЕ, КАК ИМЕННО АГЕНТ ОШИБАЕТСЯ НА EVAL-МНОЖЕСТВЕ, ДЛЯ ЭТОГО ЕСТЬ TRAIN**.
5. Другие идеи (опционально):
- с помощью нейросетевого ретривала можно найти в обучающем множестве похожие размеченные примеры
- можно реализовать tool для парсинга сайта, которую агент будет вызывать

## Про платные API

Обратите внимание, что для реализации проекта потребуется самостоятельно ходить в OpenAI API (и другие платные API). На это может уйти существенная сумма (по грубой оценке ментора, 500 запусков агента на стеке OpenAI легко уложатся в 2к рублей, но риски вылезти за эту границу есть. Можно использовать и более дешевые LLM). Также поможет иностранная карта для оплаты этих сервисов, но в целом без нее можно обойтись.

Сам я для запросов в OpenAI API часто использую сервис https://vsegpt.ru/ (не реклама). При их использовании вам не нужен VPN, достаточно российской карты, а цены не сильно отличаются от цен OpenAI, но нужно оплатить подписку (400 рублей).

**О ТОМ, КАКИЕ КЛАССНЫЕ И ДЕШЕВЫЕ АЛЬТЕРНАТИВЫ ВЫ НАШЛИ, ОБЯЗАТЕЛЬНО НАПИШИТЕ В ЧАТ**

## Система оценивания

1. Реализация бейзлайна (без использования агента, но можно с использованием LLM) -- 2 балла
2. Реализация "какого-то" агента, который под капотом имеет возможность обращаться к поисковой строке для уточнения результата -- 4 балла
3. Существенно побить бейзлайн или безуспешно приложить к этому усилия и "показать", что агенты в этой задаче прироста не дают -- 4 балла
4. (опционально, за шоколадку) реализовать одну из доп. идей по улучшению агента

## Что почитать/посмотреть по теме

1. Статья, по которой можно составить какое-то представление об агенте -- https://habr.com/ru/articles/864646/
2. https://www.youtube.com/watch?v=U6LbW2IFUQw -- неплохое видео, где на практике подробно расписано, как агента можно создавать
3. Придётся самим много читать и разбираться. Делитесь идеями в чате:)

## Описание и загрузка данных

Данные: https://disk.yandex.ru/d/6d5hFHvpAZjQdw

Ваша задача -- предсказать колонку relevance, используя все остальные данные об организации. Загрузим данные и посмотрим на них

In [1]:
import pandas as pd

data = pd.read_json(path_or_buf="./content/data_final_for_dls_new.jsonl", lines=True)
data.rename(columns= {"relevance": "relevance_old", "relevance_new": "relevance"}, inplace=True)
data

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance_old,reviews_summarized,relevance
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0
3,спортзал где 1 занятие бесплатно,"Москва, Страстной бульвар, 13А",Унца Унца Спорт; Unza Unza Sport,Фитнес-клуб,201938477844,Фитнес-клуб предлагает пробные занятия по разл...,0.1,Организация «Унца Унца Спорт» предоставляет ус...,0.1
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0
...,...,...,...,...,...,...,...,...,...
35089,нотариус запись,"Москва, 15-я Парковая улица, 45",Нотариус О. Н. Савина; Notarius O. N. Savina; ...,Нотариусы,1056199530,None,1.0,Организация предоставляет нотариальные услуги ...,1.0
35090,стационар для кота москва,"Москва, улица Госпитальный Вал, 3, корп. 5",ТриоВет; Triovet; Триовет; Veterinaria Triovet,Ветеринарная клиника,133156701339,"Ветеринарная клиника, аптека и лаборатория «Тр...",0.0,"Организация занимается ветеринарными услугами,...",0.0
35091,агзс пропан,"Самара, улица 22-го Партсъезда, 49, корп. 1",Роза Мира; АЗС № 2; Роза мира,АЗС,6884296946,None,0.0,Организация занимается заправкой транспортных ...,0.0
35092,где вибрить ваз 2112,"Нижний Новгород, Московское шоссе, 34","Нижегородец, Lada; Nizhegorodec, Lada; Нижегор...",Автосалон,124836381099,Автосалон «Нижегородец» предлагает новые автом...,0.0,Автосалон «Нижегородец» занимается продажей и ...,0.0


Здесь 1.0 соответствует оценке RELEVANT_PLUS, 0.1 -- оценке RELEVANT_MINUS, 0.0 -- оценке IRRELEVANT.

Ваша задача -- построить LLM-агента, который будет предсказывать релевантность.

Выделим данные для оценки качества агента. Запуск агента -- это тяжелая и потенциально дорогая операция. Поэтому eval-множество имеет размер 500. Также для простоты из eval-множества выкинуты данные с оценкой RELEVANT_MINUS. Тем не менее, вы можете использовать такие примеры для подачи примеров агенту.

**ОБРАТИТЕ ВНИМАНИЕ, ЧТО В EVAL-ДАННЫЕ НЕЛЬЗЯ ПОДГЛЯДЫВАТЬ ДЛЯ КАЛИБРОВКИ АГЕНТА!!! ДЛЯ ЭТОГО ЕСТЬ ОБУЧАЮЩИЕ ДАННЫЕ**

В качестве метрики качества мы будем использовать обычную ACCURACY, поскольку классы сбалансированы.

In [2]:
train_data = data[570:]
train_data = train_data[train_data['relevance'] != 0.1] # добавим фильтр для консистентности трейна и теста

eval_data = data[:570]
eval_data = eval_data[eval_data["relevance"] != 0.1]
eval_data

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance_old,reviews_summarized,relevance
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,1219173871,Rostic's предлагает различные наборы быстрого ...,1.0,"Организация занимается быстрым питанием, предо...",1.0
...,...,...,...,...,...,...,...,...,...
561,наращивание ресниц,"Саратов, улица имени А.С. Пушкина, 1",Сила; Sila; Beauty brow; Студия бровей Beauty ...,Салон красоты,236976975812,Салон красоты «Сила» предлагает услуги по уход...,1.0,Организация «Сила» занимается предоставлением ...,1.0
565,игры,"Москва, Щёлковское шоссе, 79, корп. 1",YouPlay; YouPlay КиберКлуб,Компьютерный клуб,109673025161,YouPlay КиберКлуб предлагает услуги по игре на...,0.0,Организация занимается предоставлением услуг к...,0.0
566,домашний интернет в курске что подключить отзы...,"Курск, Садовая улица, 5",Цифровой канал; Digital Channel; DChannel; ЦК;...,Телекоммуникационная компания,1737991898,None,0.0,None,0.0
567,гостиница волгодонск сауна номер телефона,"Ростовская область, городской округ Волгодонск...",Поплавок; Poplavok,"База , дом отдыха",147783493467,"Предлагает размещение в различных типах жилья,...",0.0,Организация «Поплавок» предлагает услуги базы ...,0.0


In [3]:
train_data

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance_old,reviews_summarized,relevance
570,налоговая 5007,"Московская область, Королёв, улица Богомолова, 4",Налоговая служба; Межрайонная ИФНС № 2; ИФНС №...,Налоговая инспекция,1377658436,None,0.0,Организация занимается обслуживанием налогопла...,0.0
571,шугаринг Красноярск,"Красноярск, микрорайон Взлётка, улица Весны, 3",Студия красоты Дарлинг; Darling; Дарлинг,Салон красоты,1085001465,Студия красоты «Дарлинг» предоставляет услуги ...,1.0,Студия красоты «Дарлинг» предоставляет бьюти-у...,1.0
573,новогодняя ночь 2018 в ресторане,"Москва, Кутузовский проспект, 36А",Atlantica seafood; Коллекция; Коллекция Food &...,Ресторан,1187477091,Ресторан Atlantica предлагает разнообразные бл...,1.0,Организация занимается ресторанным бизнесом и ...,1.0
574,балетная школа в санкт петербурге 12лет с прож...,"Санкт-Петербург, Невский проспект, 35В",Русская национальная балетная школа Илзе Лиепа...,Школа танцев,172170646460,None,0.0,Организация занимается обучением балету и хоре...,0.0
575,солкосерил гель цена,"Москва, Симферопольский бульвар, 29, корп. 1",Столички; Stolichki; Apteki Stolichki,Аптека,226500195223,Аптека «Столички» предлагает широкий спектр ле...,1.0,Организация занимается продажей лекарств и мед...,1.0
...,...,...,...,...,...,...,...,...,...
35089,нотариус запись,"Москва, 15-я Парковая улица, 45",Нотариус О. Н. Савина; Notarius O. N. Savina; ...,Нотариусы,1056199530,None,1.0,Организация предоставляет нотариальные услуги ...,1.0
35090,стационар для кота москва,"Москва, улица Госпитальный Вал, 3, корп. 5",ТриоВет; Triovet; Триовет; Veterinaria Triovet,Ветеринарная клиника,133156701339,"Ветеринарная клиника, аптека и лаборатория «Тр...",0.0,"Организация занимается ветеринарными услугами,...",0.0
35091,агзс пропан,"Самара, улица 22-го Партсъезда, 49, корп. 1",Роза Мира; АЗС № 2; Роза мира,АЗС,6884296946,None,0.0,Организация занимается заправкой транспортных ...,0.0
35092,где вибрить ваз 2112,"Нижний Новгород, Московское шоссе, 34","Нижегородец, Lada; Nizhegorodec, Lada; Нижегор...",Автосалон,124836381099,Автосалон «Нижегородец» предлагает новые автом...,0.0,Автосалон «Нижегородец» занимается продажей и ...,0.0


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29891 entries, 570 to 35093
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Text                            29891 non-null  object 
 1   address                         29891 non-null  object 
 2   name                            29891 non-null  object 
 3   normalized_main_rubric_name_ru  29891 non-null  object 
 4   permalink                       29891 non-null  int64  
 5   prices_summarized               17659 non-null  object 
 6   relevance_old                   29891 non-null  float64
 7   reviews_summarized              28688 non-null  object 
 8   relevance                       29891 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 2.3+ MB


In [51]:
%load_ext autoreload
%autoreload 2
# перезагружать локальные модули, если они меняются (иначе придется перезагружать ядро)

from relevance_agent import RelevanceAgent
sys.path.append('../../../')
from llm_secrets import *
from copy import deepcopy
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from typing import List, Dict
import json
import requests
from datetime import timedelta
import requests_cache  # КЭШИРУЕМ ЗАПРОСЫ ЧЕРЕЗ requests!
import warnings
warnings.filterwarnings('ignore')
import tqdm   
import langchain_core

def enable_http_requests_cache():
    requests_cache.install_cache(cache_name='./http_cache', backend='sqlite', serializer=None,
                                 expire_after=timedelta(days=180), urls_expire_after=None, cache_control=False,
                                 allowable_codes=(200, 204), allowable_methods=('GET', 'POST', 'HEAD'),
                                 always_revalidate=False,
                                 ignored_parameters=(
                                     'Authorization', 'X-API-KEY', 'access_token', 'api_key', 'x-folder-id', 'JWT',
                                     'cookie', 'created'), match_headers=False, filter_fn=None, key_fn=None,
                                 stale_if_error=False)
    # requests_cache.install_cache(cache_name='./http_cache', backend='filesystem', serializer=None, expire_after=timedelta(days=180), urls_expire_after=None, cache_control=False, allowable_codes=(200,204), allowable_methods=('GET', 'POST', 'HEAD'), always_revalidate=False, 
    #                             ignored_parameters=('Authorization', 'X-API-KEY', 'access_token', 'api_key', 'x-folder-id', 'JWT','cookie', 'created'), match_headers=False, filter_fn=None, key_fn=None, stale_if_error=False)

    # (cache_name='http_cache', backend=None, serializer=None, expire_after=-1, urls_expire_after=None, cache_control=False, allowable_codes=(200,), allowable_methods=('GET', 'HEAD'), always_revalidate=False, ignored_parameters=('Authorization', 'X-API-KEY', 'access_token', 'api_key'), match_headers=False, filter_fn=None, key_fn=None, stale_if_error=False, **kwargs)
    print('requests_cache enabled')

enable_http_requests_cache()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
requests_cache enabled


In [52]:
# MODEL_NAME = 'deepseek/deepseek-chat' # таймаутит
# MODEL_NAME = 'deepseek/deepseek-chat-0324-alt-structured' # не работает tool calling
# MODEL_NAME = 'deepseek/deepseek-chat-0324-alt-fast' # не работает
# MODEL_NAME = 'deepseek/deepseek-chat-0324-alt' # не работает
# MODEL_NAME = 'deepseek/deepseek-chat-alt'  # не работает
# MODEL_NAME = 'openai/gpt-4.1' # очень дорогая
MODEL_NAME = 'openai/gpt-4.1-mini'

USE_PRICES_SUMMARIZED = True
USE_REVIEWS_SUMMARIZED = True
TRAIN_DATA_PART_SIZE = 20 # обучаемся не на всем трейне, а на его части
VALID_DATA_SIZE = 5
RANDOM_STATE = 3
ENABLE_LLM_COMMENTS = True

def add_columns(df):
    df['llm_description'] = pd.NA
    df['score'] = pd.NA
    df['comment'] = ''
    df['search_queries'] = pd.NA
    
def fill_llm_description(df):
    for i, row in df.iterrows():
        llm_description= '## Тип: ' + row['normalized_main_rubric_name_ru'] + '\n\n## Название: ' + row['name'] + '\n\n## Адрес: ' + row['address']
        if USE_PRICES_SUMMARIZED and row['prices_summarized'] != None:
            llm_description += '\n\n## Описание: ' + row['prices_summarized']
        if USE_REVIEWS_SUMMARIZED and row['reviews_summarized'] != None:
            llm_description += '\n\n## Отзывы: ' + row['reviews_summarized']

        df.at[i,'llm_description'] = llm_description
    
def fill_score(df):
    for i, row in df.iterrows():
        if row['comment'] == '':
            #relevance_agent = RelevanceAgent(model_name='deepseek/deepseek-chat', system_prompt=system_prompt)
            print('\n===',i,'===\n')
            llm_description = df.llm_description[i]
            query = df.Text[i]
            user_message = USER_MESSAGE_TEMPLATE.format(query=query, llm_description=llm_description)
            relevance = df.relevance[i]
            print(user_message)
            print('relevanсe:', relevance)
            try:
                output = relevance_agent.run_agent(user_message)
                score, comment, search_queries = relevance_agent.parse_output(output, enable_llm_comments=ENABLE_LLM_COMMENTS)
                df.at[i,'score'] = score
                df.at[i,'comment'] = comment
                df.at[i,'search_queries'] = search_queries
                df.to_csv('output/output.csv')
            except Exception as e:
                print(e)
            
        
basic_prompt = '''
        Ты – специалист по поиску организаций в интернете по широким запросам пользователей. Пользователь отправляет широкий запрос (например, "ресторан с верандой" или "романтичный джаз-бар"). Такие запросы называются "рубричными": пользователь здесь ищет не конкретную организацию, а идёт в Яндекс.Карты для поиска и выбора мест.
        
        Твоя задача -  оценить релевантность запросу конкретной организации, информация о которой приведена в сообщении пользователя. Наличие дополнительных услуг или товаров не должно снижать оценку, главное - чтобы в организации было то, что нужно пользователю. Не важно, получена информация о том, что нужно пользователю, из исходного описания или из результатов поиска. Если в запросе указано местоположение - станция метро, район, адрес, город и т.д. - это является обязательным условием релевантности.
        
        Если указано название организации - релевантной является именно эта организация, а не похожие. Например, по запросу \"экономический колледж\" организация \"налоговый колледж\" релевантной не является.
        
        Если указана конкретная услуга или товар - необходимо соответствовать этой услуге в точности. Например, запросу  "заправка газовых баллонов" не является релевантной организация с услугой "обмен газовых баллонов".
         
        Для поиска данных о компании используй инструмент TavilySearchResults. При использовании поиска ищи каждый параметр отдельным поисковым запросом.
        Продолжай пользоваться поиском до тех пор, пока не соберешь все данные.
  '''

score_prompt = basic_prompt + '''    
        Обязательно ответь ТОЛЬКО ОДНИМ ЧИСЛОМ: оценка релевантности организации запросу пользователя от 0 до 100 (100 - точно релевантна, 0 - точно не релевантна, 67 - релевантна на 67%).
        Никаких других данных в ответе быть не должно, только одно число
        '''
        

comment_and_score_prompt = basic_prompt + '''  
        Ответь JSON в формате {{"comment": комментарий, почему организация является релевантной или не релевантной, "score": оценка релевантности организации запросу пользователя от 0 до 100 (100 - точно релевантна, 0 - точно не релевантна, 67 - релевантна на 67%)}}.
        Никакий других данных в ответе быть не должно, только JSON, который можно распарсить
        '''

if ENABLE_LLM_COMMENTS:
    system_prompt = comment_and_score_prompt
else:
    system_prompt = score_prompt

relevance_agent = RelevanceAgent(model_name=MODEL_NAME, system_prompt=system_prompt)

USER_MESSAGE_TEMPLATE = '# Запрос пользователя: {query}\n\n# Описание организации:\n\n{llm_description}'


LLM: openai/gpt-4.1-mini


In [7]:
add_columns(train_data)

In [8]:
fill_llm_description(train_data)
train_data

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance_old,reviews_summarized,relevance,llm_description,score,comment,search_queries
570,налоговая 5007,"Московская область, Королёв, улица Богомолова, 4",Налоговая служба; Межрайонная ИФНС № 2; ИФНС №...,Налоговая инспекция,1377658436,None,0.0,Организация занимается обслуживанием налогопла...,0.0,## Тип: Налоговая инспекция\n\n## Название: На...,<NA>,,<NA>
571,шугаринг Красноярск,"Красноярск, микрорайон Взлётка, улица Весны, 3",Студия красоты Дарлинг; Darling; Дарлинг,Салон красоты,1085001465,Студия красоты «Дарлинг» предоставляет услуги ...,1.0,Студия красоты «Дарлинг» предоставляет бьюти-у...,1.0,## Тип: Салон красоты\n\n## Название: Студия к...,<NA>,,<NA>
573,новогодняя ночь 2018 в ресторане,"Москва, Кутузовский проспект, 36А",Atlantica seafood; Коллекция; Коллекция Food &...,Ресторан,1187477091,Ресторан Atlantica предлагает разнообразные бл...,1.0,Организация занимается ресторанным бизнесом и ...,1.0,## Тип: Ресторан\n\n## Название: Atlantica sea...,<NA>,,<NA>
574,балетная школа в санкт петербурге 12лет с прож...,"Санкт-Петербург, Невский проспект, 35В",Русская национальная балетная школа Илзе Лиепа...,Школа танцев,172170646460,None,0.0,Организация занимается обучением балету и хоре...,0.0,## Тип: Школа танцев\n\n## Название: Русская н...,<NA>,,<NA>
575,солкосерил гель цена,"Москва, Симферопольский бульвар, 29, корп. 1",Столички; Stolichki; Apteki Stolichki,Аптека,226500195223,Аптека «Столички» предлагает широкий спектр ле...,1.0,Организация занимается продажей лекарств и мед...,1.0,## Тип: Аптека\n\n## Название: Столички; Stoli...,<NA>,,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35089,нотариус запись,"Москва, 15-я Парковая улица, 45",Нотариус О. Н. Савина; Notarius O. N. Savina; ...,Нотариусы,1056199530,None,1.0,Организация предоставляет нотариальные услуги ...,1.0,## Тип: Нотариусы\n\n## Название: Нотариус О. ...,<NA>,,<NA>
35090,стационар для кота москва,"Москва, улица Госпитальный Вал, 3, корп. 5",ТриоВет; Triovet; Триовет; Veterinaria Triovet,Ветеринарная клиника,133156701339,"Ветеринарная клиника, аптека и лаборатория «Тр...",0.0,"Организация занимается ветеринарными услугами,...",0.0,## Тип: Ветеринарная клиника\n\n## Название: Т...,<NA>,,<NA>
35091,агзс пропан,"Самара, улица 22-го Партсъезда, 49, корп. 1",Роза Мира; АЗС № 2; Роза мира,АЗС,6884296946,None,0.0,Организация занимается заправкой транспортных ...,0.0,## Тип: АЗС\n\n## Название: Роза Мира; АЗС № 2...,<NA>,,<NA>
35092,где вибрить ваз 2112,"Нижний Новгород, Московское шоссе, 34","Нижегородец, Lada; Nizhegorodec, Lada; Нижегор...",Автосалон,124836381099,Автосалон «Нижегородец» предлагает новые автом...,0.0,Автосалон «Нижегородец» занимается продажей и ...,0.0,"## Тип: Автосалон\n\n## Название: Нижегородец,...",<NA>,,<NA>


In [9]:
train_data_part = deepcopy(train_data.sample(frac=1, random_state=RANDOM_STATE)[0:TRAIN_DATA_PART_SIZE]).reset_index(drop=True).fillna('')
train_data_part.relevance.value_counts()

relevance
1.0    12
0.0     8
Name: count, dtype: int64

In [10]:
valid_data = deepcopy(train_data[TRAIN_DATA_PART_SIZE+1:TRAIN_DATA_PART_SIZE+1+VALID_DATA_SIZE]).reset_index(drop=True).fillna('')
valid_data.relevance.value_counts()

relevance
1.0    4
0.0    1
Name: count, dtype: int64

In [11]:
#train_data_part = pd.read_csv('output/output.csv').fillna('')
train_data_part

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance_old,reviews_summarized,relevance,llm_description,score,comment,search_queries
0,Страховой магазин,"Москва, улица Твардовского, 8, корп. 4",Страхование; Auto Insurance; ОСАГО; Автострахо...,Страхование автомобилей,193738061023,"Предоставляет услуги страхования автомобилей, ...",1.0,Организация занимается страхованием автомобиле...,1.0,## Тип: Страхование автомобилей\n\n## Название...,,,
1,customs.ru,"Москва, проспект Мира, 119, стр. 442",Выставочный таможенный пост № 2; Moscow custom...,Таможня,1134224338,,1.0,Организация занимается таможенными процедурами...,1.0,## Тип: Таможня\n\n## Название: Выставочный та...,,,
2,недорогие кафе казани,"Республика Татарстан, Казань, улица Бурхана Ша...",У Габила; U Gabila; Кафе у Габила,Кафе,1401012941,,1.0,"Организация занимается подачей блюд в кафе, ос...",1.0,## Тип: Кафе\n\n## Название: У Габила; U Gabil...,,,
3,танкистов сауна саратов,"Саратов, Покровская улица, 67",Филин; Filin; Баня,Баня,106612682008,"Предлагает услуги бань различного типа, а такж...",1.0,Организация занимается предоставлением услуг б...,1.0,## Тип: Баня\n\n## Название: Филин; Filin; Бан...,,,
4,невропатолог,"Москва, Авиамоторная улица, 4, корп. 3",МедФорд; MedFord; Консультативно-диагностическ...,"Медцентр , клиника",1016857008,Медицинская организация «МедФорд» предоставляе...,1.0,Организация занимается медицинскими диагностич...,1.0,"## Тип: Медцентр , клиника\n\n## Название: Мед...",,,
5,суши бар в ярославле,"Ярославль, Республиканская улица, 9",Суши Good; Sushi Good,Доставка еды и обедов,155762106969,"Суши Good предлагает доставку суши, роллов, на...",0.0,Организация занимается доставкой и продажей су...,0.0,## Тип: Доставка еды и обедов\n\n## Название: ...,,,
6,кафе в калининграде недорого,"Калининград, Эпроновская улица, 6",Во-во; Bo-Bo,Кафе,27337380614,Ресторан «Во-во» предлагает разнообразные блюд...,1.0,Организация занимается предоставлением услуг п...,1.0,## Тип: Кафе\n\n## Название: Во-во; Bo-Bo\n\n#...,,,
7,новостройки москвы на карте москвы с ценами от...,"Москва, ул. Перовская",Большое Кусково; Большом Кускове; Большом Куск...,Жилой комплекс,50785937420,,1.0,Организация представляет собой жилой комплекс ...,1.0,## Тип: Жилой комплекс\n\n## Название: Большое...,,,
8,ремонт rjvgm.nthjd егорьевск,"Московская область, Егорьевск, улица Александр...",Седьмое Небо; Seventh Sky; Натяжные потолки,Потолочные системы,63839219689,,0.0,Организация занимается установкой потолков и п...,0.0,## Тип: Потолочные системы\n\n## Название: Сед...,,,
9,офтальмолог,"Ростов-на-Дону, проспект Чехова, 55",Леге Артис; Lege Artis; Глазная клиника Леге А...,"Медцентр , клиника",1055846260,Клиника «Леге Артис» предлагает услуги по корр...,1.0,Организация «Леге Артис» занимается коррекцией...,1.0,"## Тип: Медцентр , клиника\n\n## Название: Лег...",,,


In [12]:
fill_score(train_data_part)
train_data_part


=== 0 ===

# Запрос пользователя: Страховой магазин

# Описание организации:

## Тип: Страхование автомобилей

## Название: Страхование; Auto Insurance; ОСАГО; Автострахование и электронное ОСАГО; ОСАГО, Помощь Строгино; Страхование Авто; Страхование Авто, Помощь в ГАИ; Страхование автомобилей

## Адрес: Москва, улица Твардовского, 8, корп. 4

## Описание: Предоставляет услуги страхования автомобилей, оформление договоров купли-продажи и изготовление дубликатов номерных знаков | ОСАГО | договор купли-продажи авто | дубликаты номеров

## Отзывы: Организация занимается страхованием автомобилей. Тональность отзывов преимущественно положительная: пользователи хвалят скорость обслуживания, доступные цены и профессионализм сотрудников. Критикуют завышенные цены в отдельных отзывах. | 1. Хвалят быстрое и дешёвое оформление страховки | 2. Отмечают широкий спектр услуг: страховка, договор купли-продажи, постановка на учёт | 3. Критикуют неклиентоориентированность и невыполненные обещания | 4. 

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance_old,reviews_summarized,relevance,llm_description,score,comment,search_queries
0,Страховой магазин,"Москва, улица Твардовского, 8, корп. 4",Страхование; Auto Insurance; ОСАГО; Автострахо...,Страхование автомобилей,193738061023,"Предоставляет услуги страхования автомобилей, ...",1.0,Организация занимается страхованием автомобиле...,1.0,## Тип: Страхование автомобилей\n\n## Название...,100,Организация соответствует запросу 'Страховой м...,[Страхование автомобилей Москва улица Твардовс...
1,customs.ru,"Москва, проспект Мира, 119, стр. 442",Выставочный таможенный пост № 2; Moscow custom...,Таможня,1134224338,,1.0,Организация занимается таможенными процедурами...,1.0,## Тип: Таможня\n\n## Название: Выставочный та...,90,Организация 'Выставочный таможенный пост № 2' ...,[customs.ru]
2,недорогие кафе казани,"Республика Татарстан, Казань, улица Бурхана Ша...",У Габила; U Gabila; Кафе у Габила,Кафе,1401012941,,1.0,"Организация занимается подачей блюд в кафе, ос...",1.0,## Тип: Кафе\n\n## Название: У Габила; U Gabil...,90,Кафе 'У Габила' расположено в Казани и специал...,[недорогие кафе казани]
3,танкистов сауна саратов,"Саратов, Покровская улица, 67",Филин; Filin; Баня,Баня,106612682008,"Предлагает услуги бань различного типа, а такж...",1.0,Организация занимается предоставлением услуг б...,1.0,## Тип: Баня\n\n## Название: Филин; Filin; Бан...,0,Организация Филин (баня) расположена в Саратов...,[Сауна Танкистов Саратов]
4,невропатолог,"Москва, Авиамоторная улица, 4, корп. 3",МедФорд; MedFord; Консультативно-диагностическ...,"Медцентр , клиника",1016857008,Медицинская организация «МедФорд» предоставляе...,1.0,Организация занимается медицинскими диагностич...,1.0,"## Тип: Медцентр , клиника\n\n## Название: Мед...",100,Организация МедФорд на Авиамоторной является м...,[невропатолог МедФорд на Авиамоторной]
5,суши бар в ярославле,"Ярославль, Республиканская улица, 9",Суши Good; Sushi Good,Доставка еды и обедов,155762106969,"Суши Good предлагает доставку суши, роллов, на...",0.0,Организация занимается доставкой и продажей су...,0.0,## Тип: Доставка еды и обедов\n\n## Название: ...,100,Организация 'Суши Good' расположена в Ярославл...,[суши бар Ярославль]
6,кафе в калининграде недорого,"Калининград, Эпроновская улица, 6",Во-во; Bo-Bo,Кафе,27337380614,Ресторан «Во-во» предлагает разнообразные блюд...,1.0,Организация занимается предоставлением услуг п...,1.0,## Тип: Кафе\n\n## Название: Во-во; Bo-Bo\n\n#...,85,Кафе «Во-во» находится в Калининграде и предла...,[кафе недорого в Калининграде]
7,новостройки москвы на карте москвы с ценами от...,"Москва, ул. Перовская",Большое Кусково; Большом Кускове; Большом Куск...,Жилой комплекс,50785937420,,1.0,Организация представляет собой жилой комплекс ...,1.0,## Тип: Жилой комплекс\n\n## Название: Большое...,70,Жилой комплекс 'Большое Кусково' расположен в ...,[новостройки москвы цены застройщика 2019]
8,ремонт rjvgm.nthjd егорьевск,"Московская область, Егорьевск, улица Александр...",Седьмое Небо; Seventh Sky; Натяжные потолки,Потолочные системы,63839219689,,0.0,Организация занимается установкой потолков и п...,0.0,## Тип: Потолочные системы\n\n## Название: Сед...,100,Организация 'Седьмое Небо' специализируется на...,[ремонт натяжных потолков Егорьевск]
9,офтальмолог,"Ростов-на-Дону, проспект Чехова, 55",Леге Артис; Lege Artis; Глазная клиника Леге А...,"Медцентр , клиника",1055846260,Клиника «Леге Артис» предлагает услуги по корр...,1.0,Организация «Леге Артис» занимается коррекцией...,1.0,"## Тип: Медцентр , клиника\n\n## Название: Лег...",100,Организация 'Леге Артис' является офтальмологи...,[Леге Артис офтальмолог Ростов-на-Дону]


In [13]:
train_data_part.score = train_data_part.score.fillna(0)
train_data_part.score = train_data_part.score.astype('float')

In [14]:
train_data_part.score.describe()

count     20.000000
mean      73.000000
std       37.535948
min        0.000000
25%       65.000000
50%       90.000000
75%      100.000000
max      100.000000
Name: score, dtype: float64

In [16]:
def find_best_threshold(df, metrics):
    scores = df["score"]
    labels = df["relevance"]
    thresholds = scores.drop_duplicates().sort_values(ascending=True)
    
    best_threshold = None
    best_metrics = 0
    
    for threshold in thresholds:
        preds = (scores > threshold).astype(int)
        metrics_value = metrics(labels, preds)
        if metrics_value > best_metrics:
            best_metrics = metrics_value
            best_threshold = threshold
    
    print(f"Best threshold: {best_threshold}, Train metrics: {best_metrics:.3f}")
    return best_threshold, best_metrics

In [67]:
best_threshold_f1, best_train_f1 = find_best_threshold(train_data_part, f1_score)

Best threshold: 50.0, Train metrics: 0.815


In [69]:
best_threshold_accuracy, best_train_accuracy = find_best_threshold(train_data_part, accuracy_score)
threshold = best_threshold_accuracy

Best threshold: 50.0, Train metrics: 0.750


In [70]:
# add_columns(valid_data)
# fill_llm_description(valid_data)
# valid_data

In [71]:
# fill_score(valid_data)
# valid_data

In [24]:
def eval_df(df, threshold):
    scores = df["score"]
    labels = df["relevance"]
    preds = (scores > threshold).astype(int)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)    
    print(f"Accuracy: {accuracy}, F1: {f1:.3f}")
    return accuracy, f1

In [72]:
accuracy, f1 = eval_df(train_data_part, threshold)

Accuracy: 0.75, F1: 0.815


In [66]:
# accuracy, f1 = eval_df(valid_data, threshold)

In [34]:
# EVAL
add_columns(eval_data)
fill_llm_description(eval_data)

In [60]:
fill_score(eval_data)
eval_data


=== 508 ===

# Запрос пользователя: Кузовной ремонт

# Описание организации:

## Тип: Ремонт двигателей

## Название: АМС; Ams; Техцентр; ФИРМА АМС ООО ф-л; Avtoservis AMS

## Адрес: Москва, Зеленоград, проезд № 4807, 2, стр. 13

## Описание: АМС предлагает услуги по ремонту двигателей, включая расточку и гильзовку блоков цилиндров, фрезеровку головки блока цилиндров и блока, проточку тормозных дисков, а также изготовление кованых поршней и мойку деталей | расточка блока цилиндров | гильзовка блока цилиндров | проточка тормозных дисков | фрезеровка плоскости ГБЦ | капитальный ремонт ГБЦ | фрезеровка плоскости блока цилиндров | изготовление кованых поршней | мойка деталей двигателя | капитальный ремонт двигателей

## Отзывы: Организация занимается ремонтом двигателей, включая расточку блоков, гильзовку, шлифовку ГБЦ и изготовление поршней. Отзывы в основном положительные: хвалят профессионализм мастеров, качество работ и адекватную стоимость. Однако есть замечания по поводу высоких цен

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance_old,reviews_summarized,relevance,llm_description,score,comment,search_queries
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0,## Тип: Магазин табака и курительных принадлеж...,65,Организация 'Tabaccos' в Москве занимается про...,[сигары магазин табак Москва]
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0,## Тип: Кафе\n\n## Название: PioNero; Pionero;...,20,Организация PioNero расположена в Санкт-Петерб...,[кальянная мероприятия Санкт-Петербург]
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0,## Тип: Стоматологическая клиника\n\n## Назван...,100,Организация Центр красоты и здоровья MaxiLife ...,[эпиляция в центрах красоты здоровья Одинцово]
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0,## Тип: Магазин бытовой техники\n\n## Название...,100,"Организация М.Видео, расположенная в Москве на...",[М.Видео Москва стиральные машины]
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,1219173871,Rostic's предлагает различные наборы быстрого ...,1.0,"Организация занимается быстрым питанием, предо...",1.0,## Тип: Быстрое питание\n\n## Название: Rostic...,95,Организация Rostic's соответствует запросу 'се...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,наращивание ресниц,"Саратов, улица имени А.С. Пушкина, 1",Сила; Sila; Beauty brow; Студия бровей Beauty ...,Салон красоты,236976975812,Салон красоты «Сила» предлагает услуги по уход...,1.0,Организация «Сила» занимается предоставлением ...,1.0,## Тип: Салон красоты\n\n## Название: Сила; Si...,100,Организация 'Сила' расположена в Саратове и пр...,[Сила Саратов отзывы]
565,игры,"Москва, Щёлковское шоссе, 79, корп. 1",YouPlay; YouPlay КиберКлуб,Компьютерный клуб,109673025161,YouPlay КиберКлуб предлагает услуги по игре на...,0.0,Организация занимается предоставлением услуг к...,0.0,## Тип: Компьютерный клуб\n\n## Название: YouP...,100,Запрос пользователя 'игры' очень общий. Органи...,"[YouPlay КиберКлуб Москва Щёлковское шоссе, 79..."
566,домашний интернет в курске что подключить отзы...,"Курск, Садовая улица, 5",Цифровой канал; Digital Channel; DChannel; ЦК;...,Телекоммуникационная компания,1737991898,None,0.0,None,0.0,## Тип: Телекоммуникационная компания\n\n## На...,90,Организация 'Цифровой канал' расположена в Кур...,[домашний интернет в курске отзывы цены]
567,гостиница волгодонск сауна номер телефона,"Ростовская область, городской округ Волгодонск...",Поплавок; Poplavok,"База , дом отдыха",147783493467,"Предлагает размещение в различных типах жилья,...",0.0,Организация «Поплавок» предлагает услуги базы ...,0.0,"## Тип: База , дом отдыха\n\n## Название: Попл...",85,Организация 'Поплавок' является базой отдыха в...,[гостиница волгодонск сауна номер телефона]


In [73]:
accuracy, f1 = eval_df(eval_data, threshold)

Accuracy: 0.796, F1: 0.848


In [74]:
eval_data.relevance.value_counts()

relevance
1.0    317
0.0    183
Name: count, dtype: int64

In [37]:
# ПЕРВЫЕ 50 ЗАПИСЕЙ EVAL - ДЛЯ ПОДБОРА ПОРОГА (ПОСКОЛЬКУ РАЗМЕТКА ОТЛИЧАЕТСЯ ОТ ТРЕЙНА):
eval_data_part = eval_data[0:50]
fill_score(eval_data_part)
eval_data


=== 2 ===

# Запрос пользователя: Эпиляция

# Описание организации:

## Тип: Стоматологическая клиника

## Название: MaxiLife; Центр красоты и здоровья MaxiLife; Центр красоты и здоровья МаксиЛайф; MaxiLife Health and Beauty Center

## Адрес: Московская область, Одинцово, улица Маршала Жукова, 11А

## Описание: Стоматологическая клиника, массажный салон и косметологическая студия, предоставляющие широкий спектр услуг по уходу за кожей, массажу, стоматологическим и косметологическим процедурам | чистка лица | массаж тела | лазерное омоложение | установка брекетов | пилинг | плазмолифтинг | массаж лица | педикюр | маникюр | эпиляция

## Отзывы: Организация занимается стоматологическими, косметологическими и массажными услугами. Тональность отзывов преимущественно положительная: пользователи хвалят профессионализм врачей и персонала, качество услуг и приятную атмосферу. Критикуется навязчивость предложений дополнительных услуг и отдельные случаи некачественного обслуживания.

| 1. Пользо

,Text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance_old,reviews_summarized,relevance,llm_description,score,comment,search_queries
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,Магазин табака и курительных принадлежностей,1263329400,None,1.0,"Организация занимается продажей табака, курите...",1.0,## Тип: Магазин табака и курительных принадлеж...,65,Организация 'Tabaccos' в Москве занимается про...,[сигары магазин табак Москва]
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,Кафе,228111266197,PioNero предлагает разнообразные блюда итальян...,0.0,"Организация PioNero — это кафе, бар и ресторан...",0.0,## Тип: Кафе\n\n## Название: PioNero; Pionero;...,20,Организация PioNero расположена в Санкт-Петерб...,[кальянная мероприятия Санкт-Петербург]
2,Эпиляция,"Московская область, Одинцово, улица Маршала Жу...",MaxiLife; Центр красоты и здоровья MaxiLife; Ц...,Стоматологическая клиника,1247255817,"Стоматологическая клиника, массажный салон и к...",1.0,"Организация занимается стоматологическими, кос...",1.0,## Тип: Стоматологическая клиника\n\n## Назван...,100,Организация Центр красоты и здоровья MaxiLife ...,[эпиляция в центрах красоты здоровья Одинцово]
4,стиральных машин,"Москва, улица Обручева, 34/63",М.Видео; M Video; M. Видео; M.Видео; Mvideo; М...,Магазин бытовой техники,1074529324,М.Видео предлагает широкий ассортимент бытовой...,1.0,Организация занимается продажей бытовой техник...,1.0,## Тип: Магазин бытовой техники\n\n## Название...,100,"Организация М.Видео, расположенная в Москве на...",[М.Видео Москва стиральные машины]
5,сеть быстрого питания,"Санкт-Петербург, 1-я Красноармейская улица, 15",Rostic's; KFC; Ресторан быстрого питания KFC,Быстрое питание,1219173871,Rostic's предлагает различные наборы быстрого ...,1.0,"Организация занимается быстрым питанием, предо...",1.0,## Тип: Быстрое питание\n\n## Название: Rostic...,95,Организация Rostic's соответствует запросу 'се...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,наращивание ресниц,"Саратов, улица имени А.С. Пушкина, 1",Сила; Sila; Beauty brow; Студия бровей Beauty ...,Салон красоты,236976975812,Салон красоты «Сила» предлагает услуги по уход...,1.0,Организация «Сила» занимается предоставлением ...,1.0,## Тип: Салон красоты\n\n## Название: Сила; Si...,<NA>,,<NA>
565,игры,"Москва, Щёлковское шоссе, 79, корп. 1",YouPlay; YouPlay КиберКлуб,Компьютерный клуб,109673025161,YouPlay КиберКлуб предлагает услуги по игре на...,0.0,Организация занимается предоставлением услуг к...,0.0,## Тип: Компьютерный клуб\n\n## Название: YouP...,<NA>,,<NA>
566,домашний интернет в курске что подключить отзы...,"Курск, Садовая улица, 5",Цифровой канал; Digital Channel; DChannel; ЦК;...,Телекоммуникационная компания,1737991898,None,0.0,None,0.0,## Тип: Телекоммуникационная компания\n\n## На...,<NA>,,<NA>
567,гостиница волгодонск сауна номер телефона,"Ростовская область, городской округ Волгодонск...",Поплавок; Poplavok,"База , дом отдыха",147783493467,"Предлагает размещение в различных типах жилья,...",0.0,Организация «Поплавок» предлагает услуги базы ...,0.0,"## Тип: База , дом отдыха\n\n## Название: Попл...",<NA>,,<NA>


In [78]:
# выбираем трешхолд по первым 50 записям из эвала - к сожалению, придется так, тк разметка трейна отличается от эвала:
best_threshold_accuracy, best_train_accuracy = find_best_threshold(eval_data_part, accuracy_score)
threshold = best_threshold_accuracy

Best threshold: 60, Train metrics: 0.900


In [79]:
accuracy, f1 = eval_df(eval_data_part, threshold)

Accuracy: 0.9, F1: 0.928


In [80]:
def fill_result_type(df, threshold):
    for i,row in df.iterrows():
        p_n = 'P' if row['score'] >= threshold else 'N'
        t_f = 'T' if ( ( row['relevanve']==1 and p_n == 'P') or ( row['relevanve']!=1 and p_n == 'N') ) else 'F' 
        df.at[i, 'result_type'] = t_f + p_n
    return df